In [4]:
import torch
import os
import numpy 
import pandas as pd
from PIL import Image 
from transformers import AutoImageProcessor
from transformers import TimesformerModel, logging
from src.models.video.models.timesformer import TimeSformer
from src.preprocessing.video.utils import normalize_path
data = pd.read_csv("D:\\projects\\v2v\\v5\\data\\timesformer_dataset_v2.csv", sep = ";")

In [5]:
sample_test = "F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht"
# data_path = data[data['video_base_dir'] == sample_test]
sample_frames = sorted([f"{sample_test}/{i}/frames" for i in os.listdir(sample_test) \
    if not (i.endswith(".json") or i.endswith('.csv'))], \
        key = lambda x: int(x.split("/")[-2].split("_")[-1])
        )

In [6]:
import os
video = "F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht"
# os.listdir(video)

In [24]:
import math
def sample_frames_indices(video: str, method: str, spacing: int):
        list_frames = [f"{video}/{i}" for i in os.listdir(video) if not i.endswith(".json")]
        if method == 'from_top':
            start_idx = 0
            end_idx = len(list_frames) - 1
            # Giới hạn số lượng indices tối đa theo x
            actual_spacing = min(spacing, 30, len(list_frames))
            
            indices = numpy.linspace(start_idx, end_idx, actual_spacing)
            indices = numpy.clip(indices, start_idx, end_idx - 1).astype(numpy.int64)
        elif method == 'from_middle':
            center_idx = math.ceil(len(list_frames) / 2) - 1
            half_spacing = spacing // 2
            
            # Xác định khoảng lấy mẫu
            start_idx = max(0, center_idx - half_spacing)
            end_idx = min(len(list_frames) - 1, center_idx + half_spacing)
            
            # Giới hạn số lượng indices tối đa theo x
            actual_spacing = min(spacing, 30, len(list_frames))
            
            indices = numpy.linspace(start_idx, end_idx, actual_spacing)
            indices = numpy.clip(indices, 0, len(list_frames) - 1).astype(numpy.int64)
            
        frames_list = []  
        for i, j in enumerate(list_frames):
            if i in indices:
                frames_list.append(j)
        print(frames_list)
        frames_list = [Image.open(img).convert('RGB') for img in frames_list]
        
        return frames_list


vdo = sample_frames_indices(video = f"{video}/chunk_91/frames",
                      method = 'from_middle',
                      spacing = 30)    

vdo

['F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/chunk_91/frames/frame_0001.png', 'F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/chunk_91/frames/frame_0002.png', 'F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/chunk_91/frames/frame_0003.png', 'F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/chunk_91/frames/frame_0004.png', 'F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/chunk_91/frames/frame_0005.png', 'F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/chunk_91/frames/frame_0006.png', 'F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/chunk_91/frames/frame_0007.png', 'F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/chunk_91/frames/frame_0008.png', 'F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/chunk_91/frames/frame_0009.png', 'F:/video_classification/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/chunk_91/fra

[<PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=398x224>,
 <PIL.Image.Image image mode=RGB size=39

In [19]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400", use_fast = True)
inputs = image_processor(list(vdo), return_tensors="pt").to(device)

In [18]:
from torch.optim import Adam
model = TimeSformer(30, 17).to(device)
ckpt = torch.load("./ds_tv360_model_timesformer_nepochs_2_nframes_30.pth")
optimizer = Adam([{"params": model.parameters(), "lr": 0.00001}])
model.backbone.load_state_dict(ckpt["backbone"])
model.classifier.load_state_dict(ckpt["classifier"])
optimizer.load_state_dict(ckpt["optimizer"])

Some weights of TimesformerModel were not initialized from the model checkpoint at fcakyon/timesformer-large-finetuned-k400 and are newly initialized because the shapes did not match:
- timesformer.embeddings.time_embeddings: found shape torch.Size([1, 96, 768]) in the checkpoint and torch.Size([1, 30, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
threshold = 0.5
with torch.no_grad():
    outputs = model.backbone(**inputs)
    logits = model.classifier(outputs[0][:, 0])
    probabilities = torch.sigmoid(logits)
    predictions = (probabilities >= threshold).int() 
    print(probabilities)
    

tensor([[0.3371, 0.3837, 0.3741, 0.4338, 0.3416, 0.4571, 0.4195, 0.5029, 0.6915,
         0.6164, 0.6040, 0.5748, 0.4252, 0.4554, 0.4673, 0.2496, 0.2635]],
       device='cuda:0')


In [41]:
type(predictions)

torch.Tensor

In [26]:
def binary2label(values: torch.Tensor):
    return torch.nonzero(values[0], as_tuple=True)[0].tolist()
def label2binary():
    pass

def inferencing(inputs, threshold):
    with torch.no_grad():
        outputs = model.backbone(**inputs)
        logits = model.classifier(outputs[0][:, 0])
        probabilities = torch.sigmoid(logits)
        predictions = (probabilities >= threshold).int()
    return binary2label(predictions)

inferencing(inputs, threshold)  

[7, 8, 9, 10, 11]

In [2]:
import os
os.path.exists("F:/video_classification/2014-11-04 - 20-00 Zenit Petersburg 1 - 2 Bayer Leverkusen")

True

In [29]:
from src.models.video.labels import INVERSE_EVENT_DICTIONARY_V2, EVENT_DICTIONARY_V2
INVERSE_EVENT_DICTIONARY_V2.get(1)

'Kick-off'

In [1]:
from src.models.video import TimeSformerInferencing
video = "F:/video_classification/2014-11-04 - 20-00 Zenit Petersburg 1 - 2 Bayer Leverkusen"
inferencing =TimeSformerInferencing(
    num_frames= 8,
    num_classes = 17,
    game_name = 'F:/video_classification/2014-11-04 - 20-00 Zenit Petersburg 1 - 2 Bayer Leverkusen',
    checkpoint = './ds_tv360_model_timesformer_nepochs_1_nframes_8.pth')

In [2]:

## Hiện tại đang bị lặp 
inferencing.run( method='from_top', spacing = 4, threshold =0.6)

  1%|          | 1/84 [00:00<00:41,  2.01it/s]

F:/video_classification/2014-11-04 - 20-00 Zenit Petersburg 1 - 2 Bayer Leverkusen/chunk_0/frames ------ [8]


  2%|▏         | 2/84 [00:00<00:32,  2.50it/s]

F:/video_classification/2014-11-04 - 20-00 Zenit Petersburg 1 - 2 Bayer Leverkusen/chunk_1/frames ------ [8]


  4%|▎         | 3/84 [00:01<00:29,  2.70it/s]

...

 99%|█████████▉| 83/84 [00:30<00:00,  2.71it/s]

F:/video_classification/2014-11-04 - 20-00 Zenit Petersburg 1 - 2 Bayer Leverkusen/chunk_82/frames ------ [8]


100%|██████████| 84/84 [00:31<00:00,  2.70it/s]

F:/video_classification/2014-11-04 - 20-00 Zenit Petersburg 1 - 2 Bayer Leverkusen/chunk_83/frames ------ [8]


[[8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8]]